# 3 Introducing Jupyter widgets
### Make your data analysis and visualisation interactive

<hr>
<a href="./02_Geospatial_Data_Acces.ipynb"><< 2 - Geospatial data access</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./04_Jupyter_Environment.ipynb">4 - Getting to know the Jupyter environment >></a>

* Ipyleaflet
* Plotly
* Widgets

## Example - Interactive application of climate graphs

In [9]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, Marker
import ipyleaflet
import ipywidgets as widgets
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go


In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [ ]:
ee.Initialize()

### Process mean precipitation for each month based on entire time series

In [7]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = collection.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    # if reducer function is applied to an image collection, the output does not have any projection information, as collection can contain \ 
    # images with different projection information. Thus, one can set the projection to each image
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
img_list

NameError: name 'collection' is not defined

### Build an Image Collection of the resulting image list

In [ ]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
meanMonths_collection.getInfo()

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [5]:
img = GetTileLayerUrl(meanMonths_collection.first())
img

NameError: name 'GetTileLayerUrl' is not defined

In [6]:
map1 = ipyleaflet.Map(
    zoom=4,
    layout={'height':'500px'},
)

map1.add_layer(ipyleaflet.TileLayer(url=img))
control = FullScreenControl()
map1.add_control(control)

map1

NameError: name 'img' is not defined

In [1]:
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        global lat, lon
        mark = ipyleaflet.Marker(location=kwargs.get('coordinates'))
        map1.add_layer(mark)
        location = mark.location
        lat, lon = location[0], location[1]        

map1.on_interaction(handle_click)

NameError: name 'map1' is not defined

In [10]:
def click(b):
    point = ee.Geometry.Point(lon,lat)
    tp_point = meanMonths_collection.getRegion(point,500).getInfo()
        
    header = tp_point[0]
    data = tp_point[1:]
    ydata = [row[4]*1000 for row in data]

    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))

    data = [tp]
    layout = go.Layout(
    title='Total precipitation in mm',
    )

    fig = go.Figure(data=data, layout=layout)
    with out:
        clear_output(wait=True)
        display(py.iplot(fig,filename='test'))
        
out=widgets.Output()
button=widgets.Button(description='Plot climate graph')
button.on_click(click)
display(out)
display(button)


Output()

Button(description='Plot climate graph', style=ButtonStyle())

<hr>
&copy; 2019 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>